In [1]:
pip install scipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install opencv-python
import cv2
import numpy as np
from scipy import spatial
import math
import matplotlib.pyplot as plt


# Lane Detection operation for the given video using Hough Transform

In [3]:
def gray_scale(image):
    return cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

In [4]:
def gaussian_smoothing(image,kernel_dimensions=(5,5)):
    gaussian_smooth_image = cv2.GaussianBlur(image,kernel_dimensions,0) #Applying gaussian blur to smooth the image 
    return gaussian_smooth_image

In [5]:
def canny_edge_detector(image,lower_threshold,upper_threshold):  #50,150
    return cv2.Canny(image,lower_threshold,upper_threshold)

In [6]:
def ROI_image(image):
    height = image.shape[0]
    width = image.shape[1]
    image =  np.array(image, dtype=np.float32)
   
    #polygon_coords = np.array([(0, height), (0, round(height/1.2)), (round(width/2),round(height/2)),(width, height)])
                               
    polygon_coords = np.array([(0,height),(100,360),(540,300),(620,320),(width, height)]) 
    mask  = np.zeros((height, width))
    masked_image = cv2.fillPoly(mask, [polygon_coords], 255)
    masked_image = np.array(masked_image, dtype=np.float32)
    return cv2.bitwise_and(image, masked_image)

In [7]:
def hough_transformation(ROI_img,rho,theta, threshold,lines, minLineLength, maxLineGap):
    image = cv2.convertScaleAbs(ROI_img)
    lines = cv2.HoughLinesP(image, rho, theta,threshold,lines,minLineLength, maxLineGap)
    return lines

In [8]:
def get_coordinates(image, parameters):
    slope, intercept = parameters
    if slope<0:
        y1 = image.shape[0]
        y2 = int(y1*(3/5))#3/5
        x1 = np.clip(int((y1-intercept)/slope), 0, image.shape[1])
        x2 = int((y2-intercept)/slope)
    else:
        y1 = image.shape[0]
        y2 = int(y1*(3/5))
        x1 = int((y1-intercept)/slope*0.99)#*0.98
        x2 = int((y2-intercept)/slope)
    
    return np.array([x1, y1, x2, y2])

In [9]:
def average_slope_intercept(image,lines):
    left_lines = []
    right_lines = []
    
    #getting slope and intercept for each line
    for line in lines:
        x1, y1, x2, y2 = line[0][:]
        params = np.polyfit((x1,x2), (y1,y2), 1)
        slope = params[0]
        intercept = params[1]
        
        #left and right lines seperation
        if slope<0:    #points on the left have negative slope
            left_lines.append((slope, intercept)) # so we add these points to our left_lines array
        else:
            right_lines.append((slope, intercept)) #else these points are on right side
    
    #getting coordinates for left lines
    left_line_coords = []
    for line in left_lines:
        if line[0]>-0.1:
            continue
        left_line_coords.append(get_coordinates(image, line))
    
    #getting coordinates for rigth lines
    right_line_coords = []
    for line in right_lines:
        if line[0]<0.2:
            continue
        right_line_coords.append(get_coordinates(image, line))
        
    final_left_lines = []
    final_right_lines = []
    cosine_dist = []
    #if both curves have same slope
    if len(left_line_coords)==0 or len(right_line_coords)==0:
        if len(left_line_coords)==0:
            lines = right_line_coords
        else:
            lines = left_line_coords

        if len(lines)<=2:
            return np.array(lines)
        
        for line1 in lines:
            for line2 in lines:
                cosine_dist.append(spatial.distance.cosine(line1, line2))
        cosine_dist = [item for item in cosine_dist if not(math.isnan(item))]
        threshold = np.percentile(cosine_dist,75)#80
        
        for left in lines:
            for right in lines:                    
                dist = spatial.distance.cosine(left, right)

                if dist>threshold and len(final_left_lines)==0:
                    final_left_lines.append(list(left))
                    final_right_lines.append(list(right))
                    continue
                if dist>threshold:
                    if list(left) not in final_left_lines and list(left) not in final_right_lines:
                        final_left_lines.append(list(left))
                    if list(right) not in final_right_lines and list(right) not in final_right_lines:
                        final_right_lines.append(list(right))

        left_line = np.average(np.array(final_left_lines), axis=0)
        right_line = np.average(np.array(final_right_lines), axis=0)
        return np.array([left_line, right_line])
    else:
        #keeping farthest lines and removing others
        for left in left_line_coords:
            for right in right_line_coords:
                cosine_dist.append(spatial.distance.cosine(left, right))
        cosine_dist = [item for item in cosine_dist if not(math.isnan(item))]
        threshold = np.percentile(cosine_dist,90)#85

        for left in left_line_coords:
            for right in right_line_coords:
                dist = spatial.distance.cosine(left, right)

                if dist>threshold and len(final_left_lines)==0:
                    final_left_lines.append(list(left))
                    final_right_lines.append(list(right))
                    continue
                if dist>threshold:
                    if list(left) not in final_left_lines:
                        final_left_lines.append(list(left))
                    if list(right) not in final_right_lines:
                        final_right_lines.append(list(right))

        left_line = np.average(np.array(final_left_lines), axis=0) # Axis should be 0 because we want averages of columns in array
        right_line = np.average(np.array(final_right_lines), axis=0)

        return np.array([left_line, right_line])

In [10]:
def draw_lane_lines(image, lines, color=[0,250,0], thickness=12):
    line_image = np.zeros_like(image)
    for line in lines:
        x1, y1, x2, y2 = line
        if line is not None:
            cv2.line(line_image, (int(x1),int(y1)), (int(x2),int(y2)), color, thickness)
            
            # Resultant weighted image is calculated as follows: original_img * α + img * β + γ
    return cv2.addWeighted(image, 0.8, line_image, 1, 1.0) 

In [11]:
def detect_lane(image):
    #converting to gray image
    gray_img = gray_scale(image)
    
    #gaussian smoothing
    smoothed_img = gaussian_smoothing(gray_img)
    
    #edge detection
    canny_img =canny_edge_detector(smoothed_img,50,180)
    
    #getting region on interest
    roi_img = ROI_image(canny_img)
    
    #getting line using hough transformation
    hough_lines  = hough_transformation(roi_img,1.2,np.pi/180,50,np.array([]),minLineLength=50,maxLineGap=1000)#0
    
    #generating single lines for left and right lane
    lines = average_slope_intercept(roi_img, hough_lines)
    threshold = spatial.distance.cosine(lines[0], lines[1])

    #marking lanes on the original image
    lane_marked = draw_lane_lines(image, lines)
    
    return threshold, lane_marked

In [12]:
video = cv2.VideoCapture("Lane_video.mp4")

#checking fps
fps = video.get(5)
print('Frames per second : ', fps,'FPS')

#checking total number of frames
frame_count = video.get(7)
print('Frame count : ', frame_count)

#obtaining frame size information
frame_width = int(video.get(3))
frame_height = int(video.get(4))
frame_size = (frame_width,frame_height)

#writing lane marked images into a video file
output = cv2.VideoWriter('Lane_marked_output.avi',cv2.VideoWriter_fourcc('M','J','P','G'),20, frame_size)

#loading each frame
while(video.isOpened()):
    ret, frame = video.read()
    if ret == True:
        threshold, final_image = detect_lane(frame)  
        if 0.2<=threshold<=0.303:
            output.write(final_image)
            cv2.imshow("output",final_image)
            k = cv2.waitKey(20)
            if k ==113:
                break
        else:
            continue
    else:
        break

#Releasing the video capture object and destroying all opened windows
video.release()
cv2.destroyAllWindows()

Frames per second :  30.0 FPS
Frame count :  390.0
